# Azure AI Search Memory Integration Demo
 This notebook demonstrates the integration of Azure OpenAI, Azure AI Search, and Mem0.ai to showcase different types of memory capabilities for AI agents.

In [ ]:
!pip install openai 
!pip install azure-search-documents==11.5.2
!pip install mem0ai --pre
!pip install python-dotenv

In [1]:
import os
import pandas as pd
from dotenv import load_dotenv
from mem0 import Memory, MemoryClient
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential

In [2]:
# Load environment variables
load_dotenv()

# Azure OpenAI Configuration
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME = os.getenv("AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME")
AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME")

# Azure AI Search Configuration
SEARCH_SERVICE_ENDPOINT = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
SEARCH_SERVICE_API_KEY = os.getenv("AZURE_SEARCH_ADMIN_KEY")
INDEX_NAME = "aisearch-mem0ai"

# Mem0.ai Configuration
MEM0AI_API_KEY = os.getenv("MEM0AI_API_KEY")

In [3]:
# Initialize Mem0 with Azure OpenAI and Azure AI Search integration
config = {
    "llm": {
        "provider": "azure_openai",
        "config": {
            "model": AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME,
            "temperature": 0.1,
            "max_tokens": 2000,
            "azure_kwargs": {
                "azure_deployment": AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME,
                "api_version": "2024-10-21",
                "azure_endpoint": AZURE_OPENAI_ENDPOINT,
                "api_key": AZURE_OPENAI_API_KEY,
            },
        },
    },
    "embedder": {
        "provider": "azure_openai",
        "config": {
            "model": AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME,
            "embedding_dims": 1536,
            "azure_kwargs": {
                "api_version": "2024-10-21",
                "azure_deployment": AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME,
                "azure_endpoint": AZURE_OPENAI_ENDPOINT,
                "api_key": AZURE_OPENAI_API_KEY,
            },
        },
    },
    "vector_store": {
        "provider": "azure_ai_search",
        "config": {
            "service_name": "fsunavala-performance-test",
            "api_key": SEARCH_SERVICE_API_KEY,
            "collection_name": "mem0",
            "embedding_model_dims": 1536,
            "use_compression": True,
        },
    },
    "version": "v1.1",  
}

memory = Memory.from_config(config)
mem0_client = MemoryClient(api_key=MEM0AI_API_KEY)

# We'll show two ways: 
1) `Memory` object for adding & searching in an embedded vector store,
 2) `MemoryClient` for calling advanced endpoints (e.g., exports) in hosted Mem0.

In [5]:
print("Initializing Mem0...")

# (A) Memory object
config = {
    "llm": {
        "provider": "azure_openai",
        "config": {
            "model": AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME,
            "temperature": 0.1,
            "max_tokens": 2000,
            "azure_kwargs": {
                "azure_deployment": AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME,
                "api_version": "2024-10-21",
                "azure_endpoint": AZURE_OPENAI_ENDPOINT,
                "api_key": AZURE_OPENAI_API_KEY,
            },
        },
    },
    "embedder": {
        "provider": "azure_openai",
        "config": {
            "model": AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME,
            "embedding_dims": 1536,
            "azure_kwargs": {
                "api_version": "2024-10-21",
                "azure_deployment": AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME,
                "azure_endpoint": AZURE_OPENAI_ENDPOINT,
                "api_key": AZURE_OPENAI_API_KEY,
            },
        },
    },
    "vector_store": {
        "provider": "azure_ai_search",
        "config": {
            "service_name": "fsunavala-performance-test",
            "api_key": SEARCH_SERVICE_API_KEY,
            "collection_name": "mem0",
            "embedding_model_dims": 1536,
            "use_compression": True,
        },
    },
    "version": "v1.1",  
}
memory = Memory.from_config(config)
print("Mem0 'Memory' instance created with v1.1.\n")

# (B) MemoryClient for Mem0 API endpoints
# If you want to leverage the advanced "export" endpoints, you'll need the Mem0 platform
mem0_client = MemoryClient(
    api_key=MEM0AI_API_KEY,
    # org_id="YOUR_ORG_ID",       # optional
    # project_id="YOUR_PROJECT_ID" # optional
)
print("Mem0 'MemoryClient' instance created.\n")

Initializing Mem0...
Mem0 'Memory' instance created with v1.1.

Mem0 'MemoryClient' instance created.



 ## Scenario #A USER MEMORY (Long-term memory tied to a user)

In [8]:
print("\n--- SCENARIO A) USER MEMORY ---")

# We'll store some details about a user named 'alex'
message_user = "Hi, I'm Alex. I love vegetarian food and I'm allergic to peanuts."
memory.add(message_user, user_id="alex")

message_user2 = (
    "I'm planning a trip to Japan next month. Need vegetarian-friendly places."
)
memory.add(message_user2, user_id="alex")

# Let's search for 'peanuts' to see if the memory surfaces the allergy info
search_query = "What do we know about Alex's food restrictions?"
print(f"\nSearching user memory for: {search_query}")


search_results = memory.search(
    "What do we know about Alex's food restrictions?", user_id="alex"
)
for result in search_results["results"]:
    print(f"ID: {result['id']}")
    print(f"Memory: {result['memory']}")
    print(f"Score: {result['score']}")
    print("---")


--- SCENARIO A) USER MEMORY ---

Searching user memory for: What do we know about Alex's food restrictions?
ID: 59675050-664f-456d-9e82-e67372d05da4
Memory: Name is Alex
Score: 0.639441
---
ID: 94a9aba7-1b9d-43d0-918a-43fb043753df
Memory: Loves vegetarian food
Score: 0.6141225
---
ID: 4dd6930f-3cb2-4355-844c-5bd19b4be963
Memory: Allergic to peanuts
Score: 0.6106853
---
ID: 0bf81d0f-e104-4089-88b7-924223b9d892
Memory: Need vegetarian-friendly places
Score: 0.5960605
---
ID: 3106cae6-a950-485c-8406-596a97052a92
Memory: Planning a trip to Japan next month
Score: 0.52538854
---


## Scenario B) SESSION MEMORY (Short-term memory tied to a run_id)


In [10]:
print("\n--- SCENARIO B) SESSION MEMORY ---")

session_id = "alex-trip-2024"

message_session = "During this session, I'm focusing on Tokyo hotels."
memory.add(message_session, user_id="alex", run_id=session_id)

message_session2 = "Ensure I have a 3-star or better hotel rating."
memory.add(message_session2, user_id="alex", run_id=session_id)

search_query = "What is the hotel rating preference?"
print(f"\nSearching session memory for: {search_query}")
results_session_mem = memory.search(search_query, user_id="alex", run_id=session_id)

print("Session Memory Search Results:")
for result in results_session_mem["results"]:
    print(f"ID: {result['id']}")
    print(f"Memory: {result['memory']}")
    print(f"Score: {result['score']}")
    print("---")


--- SCENARIO B) SESSION MEMORY ---

Searching session memory for: What is the hotel rating preference?
Session Memory Search Results:
ID: b4acf876-97f5-4c0a-9b97-81ee2edcdd62
Memory: Prefers hotels with a 3-star or better rating
Score: 0.7724846
---
ID: 8751751c-d2b5-485c-923c-eb9b2dc120ba
Memory: Focusing on Tokyo hotels during this session
Score: 0.6007692
---
ID: 0bf81d0f-e104-4089-88b7-924223b9d892
Memory: Need vegetarian-friendly places
Score: 0.55432814
---
ID: 94a9aba7-1b9d-43d0-918a-43fb043753df
Memory: Loves vegetarian food
Score: 0.549034
---
ID: 3106cae6-a950-485c-8406-596a97052a92
Memory: Planning a trip to Japan next month
Score: 0.5437349
---
ID: 59675050-664f-456d-9e82-e67372d05da4
Memory: Name is Alex
Score: 0.530829
---
ID: 4dd6930f-3cb2-4355-844c-5bd19b4be963
Memory: Allergic to peanuts
Score: 0.5198238
---


## Scenario C) AGENT/ASSISTANT MEMORY (Long-term memory tied to an agent)

In [11]:
print("\n--- SCENARIO C) AGENT/ASSISTANT MEMORY ---")

agent_id = "travel-assistant"

agent_context = "I am a travel planning AI with a friendly and professional personality."
memory.add(agent_context, agent_id=agent_id)

agent_update = "I provide location recommendations based on user dietary restrictions."
memory.add(agent_update, agent_id=agent_id)

search_query = "What do you do regarding dietary restrictions?"
print(f"\nSearching agent memory for: {search_query}")
results_agent_mem = memory.search(search_query, agent_id=agent_id)

print("Agent Memory Search Results:")
for result in results_agent_mem["results"]:
    print(f"ID: {result['id']}")
    print(f"Memory: {result['memory']}")
    print(f"Score: {result['score']}")
    print("---")

print("\nAll done with the main scenarios!")


--- SCENARIO C) AGENT/ASSISTANT MEMORY ---

Searching agent memory for: What do you do regarding dietary restrictions?
Agent Memory Search Results:
ID: 0bf81d0f-e104-4089-88b7-924223b9d892
Memory: Need vegetarian-friendly places
Score: 0.6403615
---
ID: 94a9aba7-1b9d-43d0-918a-43fb043753df
Memory: Loves vegetarian food
Score: 0.6259839
---
ID: 4dd6930f-3cb2-4355-844c-5bd19b4be963
Memory: Allergic to peanuts
Score: 0.62211466
---
ID: b4acf876-97f5-4c0a-9b97-81ee2edcdd62
Memory: Prefers hotels with a 3-star or better rating
Score: 0.5456611
---
ID: 3106cae6-a950-485c-8406-596a97052a92
Memory: Planning a trip to Japan next month
Score: 0.5433131
---
ID: 70f1a30e-d836-4170-80d6-f32ff9e6d60a
Memory: Has a friendly and professional personality
Score: 0.53741926
---
ID: 59675050-664f-456d-9e82-e67372d05da4
Memory: Name is Alex
Score: 0.53301156
---
ID: fcf1a741-0d2b-4592-9034-baa3049824c7
Memory: Is a travel planning AI
Score: 0.53050303
---
ID: 8751751c-d2b5-485c-923c-eb9b2dc120ba
Memory: F

## 2) Multi-Agent Coorindation

In [16]:
print("=== (2) Multi-Agent Coordination ===")

common_message = (
    "Customer has peanut allergy. Both travel and dining recommendations must exclude peanuts."
)

# We can store an array of agent_ids (if Mem0 environment supports multi-IDs).
# If not, you can loop over the agent IDs. We'll demonstrate the array approach here:
agents = ["travel-assistant", "dining-assistant"]

# This memory is "high" priority and "safety" category
memory.add(
    messages=common_message,
    agent_id=agents,  # multi-agent coordination
    metadata={"priority": "high", "category": "safety"}
)

# Let's search from the perspective of "dining-assistant"
search_query = "How should I handle peanut allergy?"
dining_search = memory.search(query=search_query, agent_id="dining-assistant")
print("Dining Assistant sees:\n", dining_search)

# Let's also confirm "travel-assistant" sees the same memory
travel_search = memory.search(query=search_query, agent_id="travel-assistant")
print("\nTravel Assistant sees:\n", travel_search)

=== (2) Multi-Agent Coordination ===
Dining Assistant sees:
 {'results': [{'id': '4dd6930f-3cb2-4355-844c-5bd19b4be963', 'memory': 'Customer has peanut allergy', 'hash': '49f66e3401bde4a1b687b3d88342bd25', 'metadata': {'priority': 'high', 'category': 'safety'}, 'score': 0.7997422, 'created_at': '2025-01-25T14:45:56.890221-08:00', 'updated_at': '2025-01-25T15:12:53.099514-08:00', 'user_id': 'alex', 'agent_id': ['travel-assistant', 'dining-assistant']}, {'id': '4b29af9a-93c2-4c76-ad1b-e46d55c68255', 'memory': 'Travel recommendations must exclude peanuts', 'hash': '8719a76a6b7f5e8b791a9b932e35886a', 'metadata': {'priority': 'high', 'category': 'safety'}, 'score': 0.70197695, 'created_at': '2025-01-25T15:12:53.378793-08:00', 'updated_at': '2025-01-25T15:12:53.099514-08:00', 'user_id': 'alex', 'agent_id': ['travel-assistant', 'dining-assistant']}, {'id': 'd214fb8a-cf9e-4d8f-b3cf-116a19dd1312', 'memory': 'Dining recommendations must exclude peanuts', 'hash': '7b9adc0db05f97345b46947fc3f2d8a2

 ## 3) Memory Summarization

 We'll create a large transcript and then store a summarized version of it, demonstrating how Mem0 can store a high-level summary for easy retrieval (the actual summarization step would typically use an LLM)

In [26]:
print("=== (3) Memory Summarization ===")

# Step A) We define a large transcript
large_conversation = """
User: Hello, I'm trying to plan a trip to Tokyo.
Assistant: Great! What are your interests?
User: I love art museums, sushi, and scenic walks. 
       I also have a peanut allergy, so I need a place that doesn't use peanut oil.
Assistant: Understood. We can look for local art museums like the Mori Art Museum,
           sushi spots that don't use peanuts, and scenic parks for walking.
User: Perfect. Also, I'd like a 3-star or better hotel rating. Budget is moderate.
Assistant: Let me compile a few options...
   (The conversation goes on with many lines about Tokyo neighborhoods, 
    discussion of flight times, baggage rules, potential side trips to Kyoto, 
    etc. It's quite lengthy.)
"""

# Step B) Summarize the large transcript using your favorite LLM in practice,
# but here we'll do a manual "dummy" summary:
summary_text = "Summary: Alex wants art, sushi, scenic walks, must avoid peanuts, 3-star or better hotels."

# Step C) Add the summary to Mem0 with metadata indicating it's a conversation summary
session_id = "alex-long-convo"
memory.add(
    messages=summary_text,
    user_id="alex",
    run_id=session_id,
    metadata={"type": "conversation_summary"}
)

# Step D) Search for 'peanut' within that session to see if the summary appears

search_query = "What do we know about peanuts in this conversation?"
summary_search = memory.search(query=search_query, user_id="alex", run_id=session_id)

print("Memory Summarization Search Results:")
for result in summary_search["results"]:
    print(f"ID: {result['id']}")
    print(f"Memory: {result['memory']}")
    print(f"Score: {result['score']}")
    print("---")

=== (3) Memory Summarization ===
Memory Summarization Search Results:
ID: 47182ecb-8fac-47a6-8e0f-1c93f0f67c4e
Memory: Must avoid peanuts
Score: 0.67557573
---
ID: d214fb8a-cf9e-4d8f-b3cf-116a19dd1312
Memory: Dining recommendations must exclude peanuts
Score: 0.66449577
---
ID: bd80007f-5b2c-4188-aff3-0b17ae9394f8
Memory: Travel and dining must exclude peanuts for Alex due to severe allergy
Score: 0.6206453
---
ID: 275c7daa-2ecc-4a13-9627-308dbab047d9
Memory: Provides location recommendations based on user dietary restrictions
Score: 0.554876
---
ID: 94a9aba7-1b9d-43d0-918a-43fb043753df
Memory: Loves vegetarian food
Score: 0.5472002
---
ID: fb6f3a95-f401-41d3-a614-4caeedeb29e5
Memory: Prefer to keep it private
Score: 0.5459654
---
ID: ff57bc6f-7486-43dc-a509-ff0eea1e451a
Memory: Prefers vegetarian meals
Score: 0.54473716
---
ID: 0bf81d0f-e104-4089-88b7-924223b9d892
Memory: Need vegetarian-friendly places
Score: 0.5370549
---
ID: 37597e99-4e1e-4e5b-9184-81a1dcc6cf2a
Memory: Upcoming med


## 1) Mapping to Memory Types

| **Memory Type**  | **Mem0 Scenario**            | **Key Characteristics**                                                                                                                                       | **Priority** |
|------------------|------------------------------|----------------------------------------------------------------------------------------------------------------------------------------------------------------|-------------|
| **Semantic**     | User Memory (Scenario A)      | - Long-term storage of user details (vegetarian, peanut allergy) <br>- Data persists across sessions <br>- Great for thread-specific (P0) or cross-thread (P1) personalization | P0          |
| **Episodic**     | Session Memory (Scenario B)   | - Short-term context for tasks (Tokyo hotel search) <br>- Tied to a run_id (single conversation or workflow) <br>- Allows ephemeral data that won’t pollute the user’s global memory | P1          |
| **Procedural**   | Agent Memory (Scenario C)     | - Agent’s own “behavior rules” and personality <br>- Instructions on how the agent handles user queries <br>- Flexible enough for P2 advanced logic (like adjusting operational rules on the fly) | P2          |

### Strategic Features

- **Thread-Specific Memory**: Achieved via `run_id`.  
- **Cross-Thread Memory**: Achieved via `user_id`.  
- **Entity Memory**: Achieved via `agent_id` (or user ID) to maintain distinct “personalities” or user entities.  


In [31]:
# %% [markdown]
# ## 4) Define a helper function to call OpenAI Chat
# 
# We'll keep it simple, just a single function that appends a user message and gets an assistant reply.
from openai import AzureOpenAI

conversation_log = [
    {"role": "system", "content": "You are a friendly travel & dining assistant."},
]

client = AzureOpenAI(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
    api_version="2024-12-01-preview",
)

def ask_gpt(user_message: str, temperature=0.7) -> str:
    """
    A simple helper to call OpenAI ChatCompletion.
    We'll store conversation context in 'conversation_log'.
    """
    conversation_log.append({"role": "user", "content": user_message})
    response = client.chat.completions.create(
        model=AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME,  # or gpt-4
        messages=conversation_log,
        temperature=temperature,
    )
    assistant_reply = response.choices[0].message.content
    conversation_log.append({"role": "assistant", "content": assistant_reply})
    return assistant_reply

print("ask_gpt function is ready.")


ask_gpt function is ready.


In [28]:
# %% [markdown]
# ## 5) Scenario A: User (Semantic) Memory + Access Control
# 
# We'll store facts about a user named 'alex', including a private note with metadata.

user_id = "alex"

# Public preference
memory.add(
    "I am allergic to peanuts and prefer vegetarian meals.",
    user_id=user_id,
    metadata={"access_level": "public"}
)

# Private note
memory.add(
    "I have an upcoming medical procedure on August 12, prefer to keep it private.",
    user_id=user_id,
    metadata={"access_level": "private"}
)

print("Scenario A: Added user memory with access control metadata.")

# Let's do a quick search to see what's there:

search_query = "What do we know about Alex's preferences?"
results_user = memory.search(query=search_query, user_id=user_id)

print("\nSearch results for user memory:")
for result in results_user["results"]:
    print(f"ID: {result['id']}")
    print(f"Memory: {result['memory']}")
    print(f"Score: {result['score']}")
    print("---")

Scenario A: Added user memory with access control metadata.

Search results for user memory:
ID: 59675050-664f-456d-9e82-e67372d05da4
Memory: Name is Alex
Score: 0.66363674
---
ID: bd80007f-5b2c-4188-aff3-0b17ae9394f8
Memory: Travel and dining must exclude peanuts for Alex due to severe allergy
Score: 0.6432383
---
ID: 9a317865-2981-4aab-a58e-c9824a1d38fb
Memory: Wants art
Score: 0.5893403
---
ID: fb6f3a95-f401-41d3-a614-4caeedeb29e5
Memory: Prefer to keep it private
Score: 0.5854313
---
ID: 70f1a30e-d836-4170-80d6-f32ff9e6d60a
Memory: Has a friendly and professional personality
Score: 0.58459395
---
ID: ff57bc6f-7486-43dc-a509-ff0eea1e451a
Memory: Prefers vegetarian meals
Score: 0.58282554
---
ID: 94a9aba7-1b9d-43d0-918a-43fb043753df
Memory: Loves vegetarian food
Score: 0.5753117
---
ID: e272f831-e089-408e-b76a-18fbdabb41a7
Memory: Prefers 3-star or better hotels
Score: 0.56723225
---
ID: fcf1a741-0d2b-4592-9034-baa3049824c7
Memory: Is a travel planning AI
Score: 0.56661254
---
ID: 27

In [21]:
# %% [markdown]
# ## 6) Scenario B: Session (Episodic) Memory
# 
# We'll create a session (run_id) for a short-term planning context, like booking hotels in Tokyo.

session_id = "alex-trip-2024"

# We'll store ephemeral details about the trip:
memory.add(
    "We want to find a 3-star or better hotel in Tokyo. Budget is moderate.",
    user_id=user_id,
    run_id=session_id
)

memory.add(
    "We also want to try local vegetarian sushi places near the hotel.",
    user_id=user_id,
    run_id=session_id
)

print("Scenario B: Session memory added.")

# Test searching within session memory
search_query = "What do we know about the hotel preference?"
session_results = memory.search(search_query, user_id=user_id, run_id=session_id)
print("\nSession memory search results:\n", session_results)


Scenario B: Session memory added.

Session memory search results:
 {'results': [{'id': 'b4acf876-97f5-4c0a-9b97-81ee2edcdd62', 'memory': 'Prefers hotels with a 3-star or better rating', 'hash': '6e09eccc50d358ffe73239ab2e5b2c61', 'metadata': None, 'score': 0.7013844, 'created_at': '2025-01-25T14:52:37.008169-08:00', 'updated_at': None, 'user_id': 'alex', 'run_id': 'alex-trip-2024'}, {'id': 'e272f831-e089-408e-b76a-18fbdabb41a7', 'memory': 'Prefers 3-star or better hotels', 'hash': '4593752963568d3e629c8d522f81458f', 'metadata': {'type': 'conversation_summary'}, 'score': 0.70133704, 'created_at': '2025-01-25T15:25:07.174197-08:00', 'updated_at': None, 'user_id': 'alex', 'run_id': 'alex-long-convo'}, {'id': '8751751c-d2b5-485c-923c-eb9b2dc120ba', 'memory': 'Focusing on Tokyo hotels during this session', 'hash': 'a5ec66410d4ea491086d065d8c7364a8', 'metadata': None, 'score': 0.6239311, 'created_at': '2025-01-25T14:52:33.832038-08:00', 'updated_at': None, 'user_id': 'alex', 'run_id': 'alex-

In [29]:
# %% [markdown]
# ## 7) Scenario C: Agent (Procedural) Memory
# 
# We'll define two agents: 'travel-assistant' and 'dining-assistant'.
# We'll store rules or instructions in these agent memories.
#
# We'll also demonstrate multi-agent memory.

agent_travel = "travel-assistant"
agent_dining = "dining-assistant"

# Travel agent rules
memory.add(
    "Always greet the user politely and consider flight times & hotel ratings.",
    agent_id=agent_travel,
    metadata={"priority": "high"}
)

# Dining agent rules
memory.add(
    "Offer at least 3 dining recommendations if user requests restaurants. Avoid peanuts.",
    agent_id=agent_dining,
    metadata={"priority": "high"}
)

print("Scenario C: Agent memory added for travel and dining assistants.")

# Multi-agent memory (both)
multi_text = (
    "Both travel and dining must exclude peanuts for Alex, as Alex is severely allergic."
)
memory.add(
    multi_text,
    agent_id=[agent_travel, agent_dining],  # if your Mem0 can store arrays
    metadata={"category": "safety"}
)

print("\nAdded multi-agent memory shared between travel-assistant and dining-assistant.")


Scenario C: Agent memory added for travel and dining assistants.

Added multi-agent memory shared between travel-assistant and dining-assistant.


In [32]:
# %% [markdown]
# ## 8) Summarization Example
# 
# We'll create a large transcript, ask GPT to summarize it, then store the summary in Mem0.

large_convo = """
User: I'm planning a trip to Tokyo. I'm vegetarian and have a peanut allergy.
Assistant: That sounds like an exciting trip! I’ll make sure to consider both your dietary preferences and your peanut allergy while planning. Let’s start by finding a hotel. Since you’re vegetarian, you might prefer hotels with easy access to vegetarian-friendly restaurants. Do you have any particular areas in Tokyo in mind for your stay, or would you like me to recommend options based on convenience and access to dining?
User: I’m thinking of staying in Shibuya or Shinjuku, as I’ve heard they’re central and have good transportation links.
Assistant: Both are excellent choices, with plenty of options for dining, sightseeing, and transport. Shibuya is vibrant and trendy, while Shinjuku is bustling with shopping, entertainment, and great restaurants. Let’s look for hotels with good access to both. For a vegetarian-friendly hotel, we can look at places that offer buffet breakfast options, with a variety of plant-based foods, or ones that are close to restaurants with vegetarian menus.
User: Shibuya sounds great! I’d like something with a mix of luxury and modern style. I prefer more contemporary hotels, maybe with a nice rooftop.
Assistant: I’ll look into a few options in Shibuya that match your style. A great place to start would be Shibuya Excel Hotel Tokyu, which offers modern amenities, a rooftop with great views, and proximity to the Shibuya Crossing, one of Tokyo’s most iconic spots. Another option is Cerulean Tower Tokyu Hotel, a luxury hotel with contemporary décor and a nice selection of vegetarian-friendly dining options. It’s also right by Shibuya Station, so you’ll have easy access to the rest of the city.
User: Those both sound good. Do they have vegetarian breakfast options, and are they close to vegetarian restaurants?
Assistant: Yes! Both hotels offer vegetarian-friendly breakfast options, including a selection of plant-based dishes. Cerulean Tower Tokyu Hotel has a few Japanese-style breakfast options that can be adapted for vegetarians, like tofu-based dishes. Shibuya Excel Hotel Tokyu offers a buffet with vegetarian selections, including fresh fruit, vegetables, and rice-based dishes. As for nearby vegetarian restaurants, Shibuya is home to Nagi Shokudo, a cozy, all-vegetarian cafe offering a range of delicious, nutritious meals. There’s also T’s Restaurant, located near Shibuya Station, which offers a fully vegan menu.
User: Perfect! What about dining in general? Since I have a peanut allergy, I need to be careful. Are there any places in Tokyo I should definitely avoid or places that cater well to my needs?
Assistant: That’s a really important consideration. Many restaurants in Japan may use peanuts in sauces or other dishes, particularly in certain regional cuisines. However, Tokyo has become increasingly aware of dietary restrictions, so there are plenty of safe options. I would recommend avoiding any street food stalls, as they can be tricky in terms of cross-contamination. When it comes to restaurant dining, it’s always best to check beforehand. I’ll list a few restaurants that cater well to allergies and offer vegetarian options.
User: I’d appreciate that! It’s great to know where I can eat without too much worry.
Assistant: Absolutely! Here are some great choices:
Ain Soph. Journey – This is a vegan restaurant located in Shinjuku. They focus on plant-based ingredients and are known for their allergy-friendly approach. They can accommodate peanut allergies with advanced notice, so it's worth calling ahead.
Noodle Stand Tokyo – Known for vegetarian and vegan ramen, this place offers safe options for people with nut allergies. They use no peanut oil in their ramen and will take care with cross-contamination if you let them know about your allergy.
Chaya Macrobiotics – A great option for healthy, plant-based dishes. Located in the Aoyama area, this restaurant offers a variety of macrobiotic meals, and they are very responsive to allergy concerns.
T’s Restaurant – Near Tokyo Station, this is a vegan paradise that doesn’t use peanut oil or any peanuts in their dishes. They are also known to be careful about allergy requests, so make sure to mention your peanut allergy.
Sushizanmai – As you mentioned, you’re interested in visiting a sushi place that doesn’t use peanut oil or peanuts. Sushizanmai is a reliable chain with a good reputation for catering to various dietary needs. You can let them know about your peanut allergy, and they will prepare your sushi without peanut oil. They also offer vegetarian sushi options like avocado rolls and veggie tempura.
User: This is a fantastic list! I’m particularly excited about the sushi place. I’ve always wanted to experience authentic sushi, but I’ve never been sure if it was safe with my peanut allergy.
Assistant: I completely understand! Traditional sushi is a great experience, and I’m glad we found a place that works with your dietary needs. It’s always a good idea to speak to the staff directly about your allergy, even if the restaurant is generally accommodating. In Japan, food allergies are taken very seriously, and many places are willing to make adjustments if you communicate your needs clearly.
As for other dining tips in Japan, I’d recommend using allergy cards in Japanese. These cards can help communicate your peanut allergy to restaurant staff. You can have them printed or use a translation app. It’s always better to be extra cautious, especially when dining in a foreign country.
User: That’s a great idea! I’ll definitely carry some allergy cards. I assume I can find these cards online to print out, right?
Assistant: Yes, exactly! There are several websites where you can find food allergy translation cards, and some apps even allow you to create personalized allergy cards in multiple languages, including Japanese. Websites like HappyCow or The Food Allergy Mama provide printable allergy cards. If you’re using a translation app, Google Translate works well for quick communication, but a printed card is ideal for consistency.
User: I’ll get those printed. What about non-vegetarian options? I don’t eat meat, but my travel companions do. Any recommendations for places that offer a mix of vegetarian and meat options?
Assistant: That’s a great consideration. Many restaurants in Tokyo cater to both vegetarians and meat-eaters, especially in places like Izakayas (Japanese pubs) and larger restaurants. Here are a few spots that have diverse menus:
Gyumon – A famous BBQ place in Shibuya that specializes in yakiniku (grilled meat). They offer vegetarian-friendly sides and dishes, such as grilled vegetables and tofu.
Tuskuna – A contemporary Japanese restaurant offering a range of dishes, including fish and seafood, but they also have creative vegetarian and vegan plates. It’s perfect for mixed dining preferences.
Sometaro – A cozy, traditional restaurant that specializes in okonomiyaki (Japanese savory pancakes). They serve both meat and vegetarian versions of the dish.
Myojin – A popular spot in Ginza for traditional sushi and tempura, which also offers vegetarian versions of both. You can enjoy a mix of plant-based and seafood-based options, while your companions can enjoy traditional meat sushi or tempura.
User: These places sound fantastic. Thank you so much for all your help with the travel planning and dining options. I’m really looking forward to the trip!
"""

# Step A) Summarize the large text with GPT
summary_prompt = f"Summarize this conversation in 3 sentences:\n\n{large_convo}"
summary_result = ask_gpt(summary_prompt)

# Step B) Store the summary in Mem0 as ephemeral context
memory.add(
    summary_result,
    user_id=user_id,
    run_id=session_id,
    metadata={"type": "conversation_summary"}
)

print("\nSummarization Example:")
print("Original text:\n", large_convo)
print("\nSummary:\n", summary_result)



Summarization Example:
Original text:
 
User: I'm planning a trip to Tokyo. I'm vegetarian and have a peanut allergy.
Assistant: That sounds like an exciting trip! I’ll make sure to consider both your dietary preferences and your peanut allergy while planning. Let’s start by finding a hotel. Since you’re vegetarian, you might prefer hotels with easy access to vegetarian-friendly restaurants. Do you have any particular areas in Tokyo in mind for your stay, or would you like me to recommend options based on convenience and access to dining?
User: I’m thinking of staying in Shibuya or Shinjuku, as I’ve heard they’re central and have good transportation links.
Assistant: Both are excellent choices, with plenty of options for dining, sightseeing, and transport. Shibuya is vibrant and trendy, while Shinjuku is bustling with shopping, entertainment, and great restaurants. Let’s look for hotels with good access to both. For a vegetarian-friendly hotel, we can look at places that offer buffet 

In [33]:
# %% [markdown]
# ## 9) Putting It All Together: Ask GPT with Mem0 Context
# 
# Let's pretend we want to generate a final recommendation based on:
# - Alex's known preferences (peanut-free, vegetarian)
# - The session's hotel rating requirement
# - The multi-agent rules about excluding peanuts
#
# We'll do a quick memory search and incorporate it in a GPT prompt.

search_query = "peanuts AND vegetarian"
user_memory_results = memory.search(search_query, user_id=user_id)

mem_bullets = []
if isinstance(user_memory_results, dict):
    results = user_memory_results.get("results", [])
    for r in results:
        mem_bullets.append("- " + r["memory"])
pref_context = "\n".join(mem_bullets) if mem_bullets else "No special memory found."

final_user_prompt = f"""I want dinner recommendations in Tokyo. Here's what Mem0 says about Alex:
{pref_context}

Considering the session with run_id={session_id}, 
please recommend a vegetarian, peanut-free restaurant near a 3-star or better hotel."""

assistant_reply = ask_gpt(final_user_prompt)
print("\nFinal GPT Response:\n", assistant_reply)



Final GPT Response:
 Hello Alex! I’m excited to help you find some great dining options in Tokyo that cater to your vegetarian and peanut-free requirements. Here are a few recommendations for vegetarian-friendly restaurants near 3-star or better hotels in Shibuya or Shinjuku:

1. **Ain Soph. Journey** (Shinjuku)
   - This vegan restaurant focuses on plant-based ingredients and is known for being allergy-friendly. They offer a delicious variety of dishes and can accommodate your peanut allergy with prior notice.

2. **Noodle Stand Tokyo** (Shibuya)
   - Famous for its vegetarian and vegan ramen, this restaurant uses no peanut oil in their dishes and takes care to avoid cross-contamination. It’s a great spot for a casual and fulfilling meal.

3. **T's Restaurant** (near Tokyo Station)
   - Although a bit further, it's worth mentioning as it specializes in vegan cuisine and is meticulous about avoiding peanuts. They have a diverse menu that includes creative sushi options.

4. **Chaya Ma